In [19]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import sklearn

In [20]:
# 导入数据
breast_cancer = datasets.load_breast_cancer()

In [ ]:
pca = PCA(svd_solver="full",random_state=0,n_components="mle")
data = pca.fit_transform(data)
print (pca.explained_variance_ratio_)
print(pca)

In [21]:
# 观察数据结构
print(breast_cancer.keys())
print(breast_cancer.data.shape,breast_cancer.target.shape)
print(breast_cancer.data[0])

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])
(569, 30) (569,)
[  1.79900000e+01   1.03800000e+01   1.22800000e+02   1.00100000e+03
   1.18400000e-01   2.77600000e-01   3.00100000e-01   1.47100000e-01
   2.41900000e-01   7.87100000e-02   1.09500000e+00   9.05300000e-01
   8.58900000e+00   1.53400000e+02   6.39900000e-03   4.90400000e-02
   5.37300000e-02   1.58700000e-02   3.00300000e-02   6.19300000e-03
   2.53800000e+01   1.73300000e+01   1.84600000e+02   2.01900000e+03
   1.62200000e-01   6.65600000e-01   7.11900000e-01   2.65400000e-01
   4.60100000e-01   1.18900000e-01]


### 数据标准化

In [22]:
std_x = StandardScaler()
data = std_x.fit_transform(breast_cancer.data)
print(data[0])
# 把数据分为数据集和测试集
std_X_train,std_X_test,std_y_train,std_y_test = train_test_split(data,breast_cancer.target,test_size=0.25,random_state=0)
# print(X_train.shape,y_train.shape)
# print(X_test.shape,y_test.shape)

[ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633  3.28351467
  2.65287398  2.53247522  2.21751501  2.25574689  2.48973393 -0.56526506
  2.83303087  2.48757756 -0.21400165  1.31686157  0.72402616  0.66081994
  1.14875667  0.90708308  1.88668963 -1.35929347  2.30360062  2.00123749
  1.30768627  2.61666502  2.10952635  2.29607613  2.75062224  1.93701461]


### knn模型---数据标准化之后

In [23]:
# 模型实例化并加载数据
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(std_X_train,std_y_train)
print("训练集精度：",knn.score(std_X_train,std_y_train))
print("测试集精度：",knn.score(std_X_test,std_y_test))

训练集精度： 0.983568075117
测试集精度： 0.944055944056


### knn模型---数据标准化之前

In [24]:
X_train,X_test,y_train,y_test = train_test_split(breast_cancer.data,breast_cancer.target,test_size=0.25,random_state=0)
# 模型实例化并加载数据
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
print("训练集精度：",knn.score(X_train,y_train))
print("测试集精度：",knn.score(X_test,y_test))

训练集精度： 0.957746478873
测试集精度： 0.923076923077


### 决策树

In [25]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=4,min_samples_leaf=4)
dt.fit(std_X_train,std_y_train)
print("训练集精度：",dt.score(std_X_train,std_y_train))
print("测试集精度：",dt.score(std_X_test,std_y_test))
print(dt)

训练集精度： 0.971830985915
测试集精度： 0.916083916084
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


In [26]:
# 绘制决策树
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO
import pydotplus
from io import StringIO
dot_data = StringIO()

export_graphviz(dt, out_file=dot_data,  # 绘制决策树
    feature_names=breast_cancer.feature_names,
    class_names=breast_cancer.target_names,
    filled=True, rounded=True,
    special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf("breast_cancer.pdf")

True

### 随机森林

In [27]:
from sklearn.ensemble import RandomForestClassifier
rand_forest = RandomForestClassifier(max_depth=6,min_samples_leaf=4,n_estimators=100)
rand_forest.fit(std_X_train,std_y_train)
print("训练集精度：",rand_forest.score(std_X_train,std_y_train))
print("测试集精度：",rand_forest.score(std_X_test,std_y_test))
print(rand_forest)

训练集精度： 0.983568075117
测试集精度： 0.972027972028
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


### SVM

In [28]:
from sklearn import svm
svm_classifier = svm.SVC(C=9.0, kernel='rbf', decision_function_shape='ovr', gamma=0.04)
svm_classifier.fit(std_X_train,std_y_train)
print(svm_classifier)
print("训练集精度：",svm_classifier.score(std_X_train,std_y_train))
print("测试集精度：",svm_classifier.score(std_X_test,std_y_test))

SVC(C=9.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.04, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
训练集精度： 0.990610328638
测试集精度： 0.986013986014


### sklearn中的人工神经网络库

In [29]:
# 标准化之前
X_train,X_test,y_train,y_test = train_test_split(breast_cancer.data,breast_cancer.target,test_size=0.25,random_state=1)
# 模型实例化并加载数据---单个隐含层，100个神经元
net_0 = MLPClassifier()
net_0.fit(X_train,y_train)
print(net)
# print("训练集精度：",net.score(X_train,y_train))
# print("测试集精度：",net.score(X_test,y_test))

NameError: name 'net' is not defined

In [ ]:
print("训练集精度：",net_0.score(X_train,y_train))
print("测试集精度：",net_0.score(X_test,y_test))

In [ ]:
# 标准化之后
std_x = StandardScaler()
data = std_x.fit_transform(breast_cancer.data)
print(data[0])
X_train,X_test,y_train,y_test = train_test_split(data,breast_cancer.target,test_size=0.25,random_state=0)
net = MLPClassifier()
net.fit(X_train,y_train)

In [ ]:
print("训练集精度：",net.score(X_train,y_train))
print("测试集精度：",net.score(X_test,y_test))

### pytorch编写的神经网络框架

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable

In [12]:
# 标准化之前
X_train,X_test,y_train,y_test = train_test_split(breast_cancer.data,breast_cancer.target,test_size=0.25,random_state=1)
# 使用 Sequential 定义 3 层神经网络
net = nn.Sequential(
    nn.Linear(30, 100),
    nn.ReLU(),
    nn.Linear(100, 2)
)
# 定义 loss 函数---交叉熵
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), 1e-3) # 使用随机梯度下降，学习率 0.001

NameError: name 'nn' is not defined

In [13]:
# 我们训练 10000 次
for e in range(10000):
    out = net(Variable(torch.from_numpy(X_train)).float())
    loss = criterion(out, Variable(torch.from_numpy(y_train)).long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if ((e + 1) % 1000 == 0) or e == 1:
        print('epoch: {}, loss: {}'.format(e+1, loss))

NameError: name 'net' is not defined

In [14]:
out = net(Variable(torch.from_numpy(X_train)).float())
prediction = torch.max(out, 1)[1] # 1返回index  0返回原值
pred_y = prediction.data.numpy()
target_y = y_train
accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
print("训练集预测准确率",accuracy)
out = net(Variable(torch.from_numpy(X_test)).float())
prediction = torch.max(out, 1)[1] # 1返回index  0返回原值
pred_y = prediction.data.numpy()
target_y = y_test
accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
print("测试预测准确率",accuracy)

NameError: name 'net' is not defined

In [15]:
# 标准化之后
std_x = StandardScaler()
data = std_x.fit_transform(breast_cancer.data)
X_train,X_test,y_train,y_test = train_test_split(data,breast_cancer.target,test_size=0.25,random_state=0)

In [16]:
# 使用 Sequential 定义 3 层神经网络
net = nn.Sequential(
    nn.Linear(30, 100),
    nn.ReLU(),
    nn.Linear(100, 2)
)
# 定义 loss 函数---交叉熵
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), 1e-3) # 使用随机梯度下降，学习率 0.001

NameError: name 'nn' is not defined

In [17]:
for e in range(10000):
    out = net(Variable(torch.from_numpy(X_train)).float())
    loss = criterion(out, Variable(torch.from_numpy(y_train)).long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if ((e + 1) % 1000 == 0) or e == 1:
        print('epoch: {}, loss: {}'.format(e+1, loss))

NameError: name 'net' is not defined

In [18]:
out = net(Variable(torch.from_numpy(X_train)).float())
prediction = torch.max(out, 1)[1] # 1返回index  0返回原值
pred_y = prediction.data.numpy()
target_y = y_train
accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
print("训练集预测准确率",accuracy)
out = net(Variable(torch.from_numpy(X_test)).float())
prediction = torch.max(out, 1)[1] # 1返回index  0返回原值
pred_y = prediction.data.numpy()
target_y = y_test
accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
print("测试预测准确率",accuracy)

NameError: name 'net' is not defined